<a href="https://colab.research.google.com/github/mistryamit03/Gornation-calisthenics-growth-analytics/blob/main/Gornation_Mock_Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ok then lets begin with generating data in collab using python.

In [1]:
# Generate a realistic 12-month mock dataset for GORNATION in Python
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta

np.random.seed(42)
random.seed(42)

# -----------------------------
# 1) Product catalog (realistic)
# -----------------------------
products = [
    # category, name, min_price, max_price, weight_kg (approx), size_category (for returns)
    ("Parallettes", "Wooden Parallettes", 45, 75, 2.2, "M"),
    ("Parallettes", "Steel Parallettes", 60, 95, 3.8, "L"),
    ("Bandages", "Wrist Wraps", 15, 25, 0.2, "S"),
    ("Bandages", "Elbow Support", 20, 30, 0.25, "S"),
    ("Pull Up Bars", "Doorway Pull-Up Bar", 70, 110, 4.0, "XL"),
    ("Pull Up Bars", "Outdoor Pull-Up Bar", 100, 160, 7.0, "XL"),
    ("Dip Belt", "Weighted Dip Belt", 35, 50, 0.8, "M"),
    ("Dip Bars", "Steel Dip Bars", 65, 110, 5.0, "XL"),
    ("Clothing", "Oversized Hoodie", 55, 85, 0.7, "M"),
    ("Clothing", "Training Tank", 25, 40, 0.25, "S"),
    ("Clothing", "Joggers", 45, 75, 0.6, "M"),
    ("Clothing", "Performance T-Shirt", 30, 45, 0.3, "S"),
    ("Static Bar", "Adjustable Static Bar", 130, 180, 8.0, "XL"),
    ("Workout Rings", "Wooden Gymnastic Rings", 30, 50, 1.1, "M"),
    ("Resistance Bands", "Set of 5 Resistance Bands", 25, 50, 0.9, "S"),
    ("Equipment Sets", "Beginner Home Workout Set", 110, 170, 6.0, "L"),
    ("Equipment Sets", "Advanced Athlete Set", 150, 220, 9.0, "XL"),
    ("Workout Accessories", "Athlete Backpack", 35, 60, 0.9, "M"),
    ("Workout Accessories", "Chalk Block", 12, 18, 0.3, "S"),
    ("Weight Vests", "10kg Weighted Vest", 100, 140, 10.0, "XL"),
    ("Weight Vests", "20kg Weighted Vest", 130, 170, 20.0, "XL"),
    ("Better Grip", "Liquid Chalk", 10, 20, 0.35, "S")
]
product_df = pd.DataFrame(products, columns=["Product_Category", "Product_Name", "Min_Price", "Max_Price", "WeightKG", "SizeCategory"])

# -----------------------------
# 2) Date range & seasonality
# -----------------------------
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)
days = (end_date - start_date).days + 1
dates = [start_date + timedelta(days=i) for i in range(days)]

# Seasonality factors per month (rough heuristic for calisthenics/e-comm)
month_factor = {
    1: 0.9,  2: 0.95, 3: 1.05, 4: 1.1,
    5: 1.15, 6: 1.2,  7: 1.1,  8: 1.0,
    9: 1.05, 10: 1.1, 11: 1.2, 12: 1.35  # Q4 bump
}

# Base daily orders
base_daily_orders = 15  # will scale with month_factor to produce ~6k-8k orders total

# -----------------------------
# 3) Customer base
# -----------------------------
n_customers = 2000
customer_ids = [f"C{100000+i}" for i in range(n_customers)]
customer_first_purchase = np.random.choice(dates, size=n_customers)
customer_region = np.random.choice(["Domestic", "EU", "Non-EU"], p=[0.45, 0.35, 0.20], size=n_customers)

customers = pd.DataFrame({
    "Customer_ID": customer_ids,
    "Customer_Since": customer_first_purchase,
    "Region_Default": customer_region
})

# -----------------------------
# 4) Channels & campaigns
# -----------------------------
channels = ["Instagram", "TikTok", "Google", "Organic"]
channel_weights = [0.35, 0.30, 0.20, 0.15]  # relative volume share

def random_campaign(channel, dt):
    month = dt.strftime("%b")
    mapping = {
        "Instagram": [f"IG_{month}_AthleteDrop", f"IG_{month}_Reels", f"IG_{month}_Giveaway"],
        "TikTok":    [f"TT_{month}_Creators", f"TT_{month}_Challenge", f"TT_{month}_SparkAds"],
        "Google":    [f"GG_{month}_Search", f"GG_{month}_Shopping", f"GG_{month}_Brand"],
        "Organic":   [f"ORG_{month}_Community", f"ORG_{month}_Blog", f"ORG_{month}_Direct"]
    }
    return random.choice(mapping[channel])

# Cost/traffic heuristics
avg_cpc = {"Instagram": 0.45, "TikTok": 0.30, "Google": 0.55, "Organic": 0.0}
avg_ctr = {"Instagram": 0.012, "TikTok": 0.018, "Google": 0.020, "Organic": 0.0}  # for impressions estimation

# -----------------------------
# 5) Generate orders day by day
# -----------------------------
orders = []
order_id_counter = 1000000

for dt in dates:
    # number of orders for the day with seasonality & random noise
    lam = base_daily_orders * month_factor[dt.month]
    num_orders = np.random.poisson(lam=lam)
    if num_orders == 0:
        continue

    # sample customers for the day; allow repeats
    cust_idxs = np.random.choice(n_customers, size=num_orders, replace=True)
    for idx in cust_idxs:
        order_id_counter += 1
        cust_id = customer_ids[idx]
        first_date = customers.loc[customers.Customer_ID == cust_id, "Customer_Since"].values[0]
        region_default = customers.loc[customers.Customer_ID == cust_id, "Region_Default"].values[0]
        customer_type = "New" if dt.date() == pd.to_datetime(first_date).date() else "Returning"

        # choose product
        prod = product_df.sample(1).iloc[0]
        unit_price = np.round(np.random.uniform(prod.Min_Price, prod.Max_Price), 2)
        units = np.random.choice([1, 2, 3], p=[0.80, 0.15, 0.05])
        revenue = round(unit_price * units, 2)

        # channel & campaign
        channel = np.random.choice(channels, p=channel_weights)
        campaign = random_campaign(channel, dt)

        # simulate marketing signals per order as "attributed shares"
        # clicks per order (at least 1 for paid channels)
        clicks = 0 if channel == "Organic" else np.random.choice([1,2,3], p=[0.7, 0.25, 0.05])
        # estimate impressions using ctr (avoid div by 0)
        impressions = 0 if channel == "Organic" else int(round(clicks / max(avg_ctr[channel], 1e-6)))
        # attributed ad spend ~ clicks * CPC
        ad_spend = round(clicks * avg_cpc[channel], 2)

        # inventory tracking: per order stock snapshot (simulate)
        # initialize product-level stock baselines per month for realism
        # We'll assume a notional stock and decrease with sales
        # For simplicity, simulate stock level as noisy function around category popularity
        base_stock = {
            "Clothing": 800, "Parallettes": 300, "Bandages": 600, "Pull Up Bars": 200,
            "Dip Belt": 250, "Dip Bars": 180, "Static Bar": 120, "Workout Rings": 400,
            "Resistance Bands": 500, "Equipment Sets": 150, "Workout Accessories": 450,
            "Weight Vests": 240, "Better Grip": 700
        }[prod.Product_Category]

        # decay stock across the year (rough)
        day_of_year = (dt - start_date).days + 1
        annual_sales_rate = base_stock * 1.2  # assume 120% turnover across the year on avg
        sold_so_far_est = int(annual_sales_rate * day_of_year / days)
        stock_level = max(base_stock - sold_so_far_est, 0) - units
        # reorder point: 15% of base
        reorder_point = int(base_stock * 0.15)

        # fulfillment metrics
        region = np.random.choice(["Domestic", "EU", "Non-EU"], p=[0.5, 0.3, 0.2])
        # shipping time distribution by region
        if region == "Domestic":
            ship_days = max(1, int(np.random.normal(2.2, 0.8)))
            ship_cost = round(np.random.uniform(4, 7) + prod.WeightKG * 0.4, 2)
        elif region == "EU":
            ship_days = max(2, int(np.random.normal(4.0, 1.2)))
            ship_cost = round(np.random.uniform(6, 10) + prod.WeightKG * 0.6, 2)
        else:
            ship_days = max(4, int(np.random.normal(7.0, 1.8)))
            ship_cost = round(np.random.uniform(9, 16) + prod.WeightKG * 0.9, 2)

        # Delivery status (on-time vs late) target SLA: Domestic<=3, EU<=5, Non-EU<=8
        sla = {"Domestic": 3, "EU": 5, "Non-EU": 8}[region]
        delivery_status = "On-time" if ship_days <= sla else "Late"

        # returns
        return_prob = {
            "S": 0.045, "M": 0.055, "L": 0.065, "XL": 0.08
        }[prod.SizeCategory]
        ret_flag = np.random.rand() < return_prob
        return_flag = "Yes" if ret_flag else "No"
        return_reason = None
        if ret_flag:
            reasons = ["Size", "Damaged", "Customs", "Other"]
            weights = [0.55, 0.20, 0.15, 0.10] if prod.Product_Category in ["Clothing"] else [0.25, 0.45, 0.15, 0.15]
            return_reason = np.random.choice(reasons, p=weights)

        orders.append({
            "Order_ID": f"O{order_id_counter}",
            "Order_Date": dt.date().isoformat(),
            "Customer_ID": cust_id,
            "Customer_Type": customer_type,
            "Customer_Since": pd.to_datetime(first_date).date().isoformat(),
            "Region": region,
            "Acquisition_Channel": channel,
            "Campaign_Name": campaign,
            "Impressions": impressions,
            "Clicks": clicks,
            "Adds_To_Cart": np.random.choice([1,2], p=[0.85, 0.15]) if clicks>0 else 0,
            "Ad_Spend": ad_spend,
            "Product_Category": prod.Product_Category,
            "Product_Name": prod.Product_Name,
            "Units_Sold": units,
            "Unit_Price": unit_price,
            "Revenue": revenue,
            "Stock_Level": stock_level,
            "Reorder_Point": reorder_point,
            "Delivery_Time_Days": ship_days,
            "Delivery_Status": delivery_status,
            "Return_Flag": return_flag,
            "Return_Reason": return_reason if return_reason else "",
            "Shipping_Cost": ship_cost
        })

orders_df = pd.DataFrame(orders)

# Shuffle rows to mimic real export and reset index
orders_df = orders_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Basic sanity checks
n_orders = len(orders_df)
summary = {
    "rows": n_orders,
    "date_range": (orders_df.Order_Date.min(), orders_df.Order_Date.max()),
    "revenue_total": float(orders_df["Revenue"].sum()),
    "ad_spend_total": float(orders_df["Ad_Spend"].sum()),
    "on_time_pct": round((orders_df["Delivery_Status"]=="On-time").mean()*100,2),
    "return_rate_pct": round((orders_df["Return_Flag"]=="Yes").mean()*100,2)
}
summary



{'rows': 6070,
 'date_range': ('2024-01-01', '2024-12-31'),
 'revenue_total': 552554.86,
 'ad_spend_total': 2955.95,
 'on_time_pct': np.float64(94.84),
 'return_rate_pct': np.float64(5.73)}

In [2]:
orders_df.head(10)

,Order_ID,Order_Date,Customer_ID,Customer_Type,Customer_Since,Region,Acquisition_Channel,Campaign_Name,Impressions,Clicks,...,Units_Sold,Unit_Price,Revenue,Stock_Level,Reorder_Point,Delivery_Time_Days,Delivery_Status,Return_Flag,Return_Reason,Shipping_Cost
0,O1002765,2024-06-23,C101603,Returning,2024-01-25,Domestic,Instagram,IG_Jun_AthleteDrop,83,1,...,1,153.51,153.51,63,22,2,On-time,No,,8.91
1,O1002723,2024-06-22,C100169,Returning,2024-11-26,Domestic,TikTok,TT_Jun_Creators,56,1,...,1,13.79,13.79,300,105,2,On-time,No,,5.55
2,O1005390,2024-11-28,C101534,Returning,2024-10-19,Non-EU,TikTok,TT_Nov_Challenge,56,1,...,2,49.32,98.64,-2,60,5,On-time,No,,13.76
3,O1004444,2024-10-05,C101999,Returning,2024-07-22,Domestic,Google,GG_Oct_Brand,100,2,...,1,17.12,17.12,51,90,1,On-time,No,,6.11
4,O1003134,2024-07-15,C101622,Returning,2024-12-14,Domestic,TikTok,TT_Jul_SparkAds,56,1,...,1,61.84,61.84,283,120,2,On-time,No,,4.37
5,O1004210,2024-09-22,C100745,Returning,2024-05-12,EU,Instagram,IG_Sep_AthleteDrop,83,1,...,1,37.04,37.04,102,120,3,On-time,No,,8.48
6,O1005577,2024-12-07,C100914,Returning,2024-05-05,Domestic,Google,GG_Dec_Brand,100,2,...,1,40.28,40.28,-1,37,1,On-time,No,,6.74
7,O1002351,2024-06-02,C101865,Returning,2024-03-24,Domestic,Instagram,IG_Jun_AthleteDrop,83,1,...,1,74.86,74.86,396,120,1,On-time,No,,5.38
8,O1002759,2024-06-23,C100549,Returning,2024-05-30,Domestic,Google,GG_Jun_Shopping,100,2,...,1,68.51,68.51,76,27,2,On-time,No,,7.59
9,O1000264,2024-01-20,C101857,Returning,2024-06-26,Domestic,Instagram,IG_Jan_Giveaway,83,1,...,1,28.89,28.89,747,120,2,On-time,No,,6.73


In [3]:
product_df.head(10)

,Product_Category,Product_Name,Min_Price,Max_Price,WeightKG,SizeCategory
0,Parallettes,Wooden Parallettes,45,75,2.20,M
1,Parallettes,Steel Parallettes,60,95,3.80,L
2,Bandages,Wrist Wraps,15,25,0.20,S
3,Bandages,Elbow Support,20,30,0.25,S
4,Pull Up Bars,Doorway Pull-Up Bar,70,110,4.00,XL
5,Pull Up Bars,Outdoor Pull-Up Bar,100,160,7.00,XL
6,Dip Belt,Weighted Dip Belt,35,50,0.80,M
7,Dip Bars,Steel Dip Bars,65,110,5.00,XL
8,Clothing,Oversized Hoodie,55,85,0.70,M
9,Clothing,Training Tank,25,40,0.25,S


In [4]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6070 entries, 0 to 6069
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Order_ID             6070 non-null   object 
 1   Order_Date           6070 non-null   object 
 2   Customer_ID          6070 non-null   object 
 3   Customer_Type        6070 non-null   object 
 4   Customer_Since       6070 non-null   object 
 5   Region               6070 non-null   object 
 6   Acquisition_Channel  6070 non-null   object 
 7   Campaign_Name        6070 non-null   object 
 8   Impressions          6070 non-null   int64  
 9   Clicks               6070 non-null   int64  
 10  Adds_To_Cart         6070 non-null   int64  
 11  Ad_Spend             6070 non-null   float64
 12  Product_Category     6070 non-null   object 
 13  Product_Name         6070 non-null   object 
 14  Units_Sold           6070 non-null   int64  
 15  Unit_Price           6070 non-null   f

In [5]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_Category  22 non-null     object 
 1   Product_Name      22 non-null     object 
 2   Min_Price         22 non-null     int64  
 3   Max_Price         22 non-null     int64  
 4   WeightKG          22 non-null     float64
 5   SizeCategory      22 non-null     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 1.2+ KB


In [7]:
orders_df.describe()

,Impressions,Clicks,Adds_To_Cart,Ad_Spend,Units_Sold,Unit_Price,Revenue,Stock_Level,Reorder_Point,Delivery_Time_Days,Shipping_Cost
count,6070.000000,6070.000000,6070.000000,6070.000000,6070.000000,6070.000000,6070.000000,6070.000000,6070.000000,6070.000000,6070.000000
mean,76.256507,1.161285,0.974300,0.486977,1.241351,73.489433,91.030455,163.976277,63.418287,3.247117,9.654598
std,52.187454,0.737068,0.527383,0.331902,0.524474,50.917330,77.773666,180.149584,35.628461,2.148826,4.463213
min,0.000000,0.000000,0.000000,0.000000,1.000000,10.030000,10.030000,-3.000000,18.000000,1.000000,4.120000
25%,50.000000,1.000000,1.000000,0.300000,1.000000,33.447500,36.690000,18.000000,30.000000,2.000000,6.510000
50%,83.000000,1.000000,1.000000,0.450000,1.000000,57.240000,67.860000,107.000000,60.000000,3.000000,8.500000
75%,100.000000,2.000000,1.000000,0.600000,1.000000,110.460000,131.517500,241.000000,90.000000,4.000000,11.820000
max,250.000000,3.000000,2.000000,1.650000,3.000000,219.980000,658.170000,797.000000,120.000000,13.000000,33.940000


In [8]:
product_df.describe()

,Min_Price,Max_Price,WeightKG
count,22.000000,22.000000,22.000000
mean,58.954545,89.909091,3.711364
std,42.597202,59.615362,4.840477
min,10.000000,18.000000,0.200000
25%,26.250000,46.250000,0.412500
50%,45.000000,75.000000,1.000000
75%,92.500000,132.500000,5.750000
max,150.000000,220.000000,20.000000
